# Doge

训练 [Wonderful Matrices](https://arxiv.org/abs/2407.16958) 论文中提出的 `Doge` 小型语言模型.
Doge在 Transformers 的框架基础上, 将序列变换部分的 `Multi-Head Attention` 替换为 `Inner Function Attention`, 将状态变换部分的 `MLP` 替换为 `CDMoE` . 

Train the `Doge` small language model proposed in the paper [Wonderful Matrices](https://arxiv.org/abs/2407.16958).
Doge is based on the Transformers framework, replacing the `Multi-Head Attention` in the sequence transformation part with `Inner Function Attention`, and replacing the `MLP` in the state transformation part with `CDMoE`.

![doge_architecture](./assets/doge_architecture.png)

## 下载预训练与微调数据集
## Download Pre-Training and Fine-Tuning Datasets

预训练数据集, 我们选取了 `cosmopedia-v2` 与 `chinese-cosmopedia` 这种高质量合成数据集并补充 `python-edu` 来保证模型的代码能力. 

For the pre-training dataset, we selected high-quality synthetic datasets such as `cosmopedia-v2` and `chinese-cosmopedia`, and supplemented them with `python-edu` to ensure the model's coding ability.

微调数据集, 我们选取了 `Infinity-Instruct` 的 `0625`, `7M` 与 `Gen` 子集.

For the fine-tuning dataset, we selected the `0625`, `7M`, and `Gen` subsets of `Infinity-Instruct`.

> 请注意: 由于数据集过大, 至少需要 2TB 的存储空间.

> Note: Due to the large size of the dataset, at least 2TB of storage space is required.

In [ ]:
# 填写保存路径, 缓存路径和进程数
# Padding save path, cache path and number of processes
!python scripts/download_datasets.py --save_dir S:/datasets --cache_dir S:/datasets/cache --num_proc 16

## 预处理数据集
## Preprocess Datasets

我们需要使用 `tokenizer` 将数据集转为模型可接受的 `input_ids` 与 `attention_mask`.
Doge 使用 `LlamaTokenizer` , 该 tokenizer 词表大小为 `32768` , 使用 `[INST]` 与 `[/INST]` 标记指令. 它还包括工具标记, 但是我们不会在这里使用它们.
像 cosmopedia-v2 与 Infinity-Instruct 这样的数据集就包括 `prompt` 与 `text` 两个字段, 我们就将他们标记为用户指令提示与模型输出文本.

We need to use the `tokenizer` to convert the dataset into `input_ids` and `attention_mask` that the model can accept.
Doge uses the `LlamaTokenizer`, which has a vocabulary size of `32768`, and uses the `[INST]` and `[/INST]` tags to mark instructions. It also includes utility tokens, but we won't use them here.
Datasets like cosmopedia-v2 and Infinity-Instruct include two fields, `prompt` and `text`, which we will mark as user instruction prompts and model output text.

```python
prompt = f"[INST]{prompt}[/INST]"
return tokenizer(prompt, text, padding='max_length', truncation=True, max_length=MAX_LENGTH)
```

当然你也可以自行加入一些指令提示.

Of course, you can also add some instruction prompts yourself.

```python
prompt = f"[INST]You are an AI assistant named `Doge`, you are a language model trained by `Shi Jingze` based on the `Doge` architecture, and your task is to provide appropriate replies and support to users based on their questions and requests.\n你是一个名为 `Doge` 的人工智能助手, 你是由 `石竞泽` 基于 `Doge` 架构训练的语言模型, 你的任务是针对用户的问题和要求提供适当的答复和支持.\n[/INST][INST]{prompt}[/INST]"
return tokenizer(prompt, text, padding='max_length', truncation=True, max_length=MAX_LENGTH)
```

In [ ]:
# 填写数据集路径, 保存路径, 分词器路径, 最大长度和进程数
# Padding dataset path, save path, tokenizer path, max length and number of processes
!python scripts/preprocess_datasets.py --datasets_dir S:/datasets --save_dir S:/datasets --tokenizer_path ./tokenizer --max_len 2048 --num_proc 16

## 合并数据集
## Concatenate Datasets

我们将 cosmopedia-v2, chinese-cosmopedia 和 python-edu 数据集合并为 `pretrain` 数据集, 将 0625, 7M 和 Gen 数据集合并为 `finetune` 数据集.
然后将它们打乱顺序 `seed=233` , 并拆分出来 `1,000` 个样本作为测试集.

We combine the cosmopedia-v2, chinese-cosmopedia, and python-edu datasets into the `pretrain` dataset, and the 0625, 7M, and Gen datasets into the `finetune` dataset.
Then shuffle the order `seed=233`, and split out `1,000` samples as the test set.

In [ ]:
# 填写数据集路径, 保存路径和进程数
# Padding dataset path, save path and number of processes
!python scripts/concatenate_datasets --datasets_dir S:/datasets --save_dir S:/datasets --num_proc 16

## 配置模型参数
## Configure Model Parameters

我们配置一个 `25M` 的小型模型, 进行训练测试.

| Params | n_layers | d_model | n_heads | n_inner_v | d_cross_domain | d_expert | n_exprets | n_expert_heads | n_expert_pre_head |
|--------|----------|---------|---------|-----------|----------------|----------|-----------|----------------|-------------------|
| 25M    | 8        | 256     | 4       | 2         | 1024           | 256      | 256       | 2              | 4                 |
| 320M   | 16       | 1024    | 8       | 8         | 4096           | 1024     | 1024      | 2              | 4                 |

- n_layers 是模型的解码器层数
- d_model 是模型的隐藏层维度
- n_heads 是InnerFuncAttn的多头注意力头数 d_model // n_heads 最好保持在 64 以上
- n_inner_v 是InnerFUncAttn的V的数量 d_model // n_inner_v 最好保持在 64 以上

In [ ]:
from yaml import safe_load

# 读取配置文件, 请根据实际情况自行修改
# Read the configuration file, please modify it according to the actual situation
with open('./model/config/doge_25M.yaml', 'r', encoding='utf-8') as f:
    config = safe_load(f)

config['model']

## 配置预训练超参数
## Configure Pre-Training Hyperparameters

| Params | log_steps | max_steps | eval_steps | save_steps | accumulate_steps | learning_rate | warmup_ratio | weight_decay | min_lr_rate | scheduler_type |
|--------|-----------|-----------|------------|------------|------------------|---------------|--------------|--------------|-------------| ---------------|
| 25M    | 100       | 2,000     | 100        | 100        | 256              | 8e-4          | 0.1          | 0.01         | 0.1         | cosine_with_min_lr |
| 320M   | 100       | 25,000    | 500        | 500        | 512              | 3e-4          | 0.1          | 0.01         | 0.1         | cosine_with_min_lr |


In [ ]:
!python scripts/train.py --config_path ./model/config/doge_25M.yaml

## 评估
## Evaluation

我们先安装 `miniconda` .

First, install `miniconda`.

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
```

然后创建评估环境.

Then create an evaluation environment.

```bash
conda create -n lighteval python=3.10.12 
conda activate lighteval
pip install lighteval[accelerate]
```

最后我们运行评估单元格.

Finally, we run the evaluation cell.


In [ ]:
MODEL = "LoserCheems/Doge-25M"
OUTPUT_DIR = "./lighteval_results"
!lighteval accelerate \
--model_args="pretrained=$MODEL" \
--output_dir $OUTPUT_DIR \
--override_batch_size 16 \
--tasks "original|mmlu|0|1,lighteval|triviaqa|0|1,lighteval|arc:easy|0|1,lighteval|piqa|0|1,leaderboard|hellaswag|0|1,lighteval|openbookqa|0|1,leaderboard|winogrande|0|1"

In [ ]:
!lighteval accelerate --model_args="pretrained=LoserCheems/Doge-25M" --output_dir "./lighteval_results" --override_batch_size 16 --tasks "original|mmlu|0|1,lighteval|piqa|0|1,leaderboard|hellaswag|0|1,lighteval|openbookqa|0|1,leaderboard|winogrande|0|1"

In [ ]:
!python lighteval/run_evals_accelerate.py --model_args="pretrained=LoserCheems/Doge-25M" --custom_tasks "lighteval_tasks.py" --output_dir ./lighteval_results --override_batch_size 16 --tasks "custom|hellaswag|0|1,custom|winogrande|0|1,custom|piqa|0|1,custom|siqa|0|1,custom|openbookqa|0|1,custom|arc:easy|0|1,custom|arc:challenge|0|1,custom|commonsense_qa|0|1,custom|trivia_qa|0|1,custom|mmlu_pro_cloze|0|1,custom|gsm8k|5|1,custom|mmlu_cloze:abstract_algebra|0|1,custom|mmlu_cloze:anatomy|0|1,custom|mmlu_cloze:astronomy|0|1,custom|mmlu_cloze:business_ethics|0|1,custom|mmlu_cloze:clinical_knowledge|0|1,custom|mmlu_cloze:college_biology|0|1,custom|mmlu_cloze:college_chemistry|0|1,custom|mmlu_cloze:college_computer_science|0|1,custom|mmlu_cloze:college_mathematics|0|1,custom|mmlu_cloze:college_medicine|0|1,custom|mmlu_cloze:college_physics|0|1,custom|mmlu_cloze:computer_security|0|1,custom|mmlu_cloze:conceptual_physics|0|1,custom|mmlu_cloze:econometrics|0|1,custom|mmlu_cloze:electrical_engineering|0|1,custom|mmlu_cloze:elementary_mathematics|0|1,custom|mmlu_cloze:formal_logic|0|1,custom|mmlu_cloze:global_facts|0|1,custom|mmlu_cloze:high_school_biology|0|1,custom|mmlu_cloze:high_school_chemistry|0|1,custom|mmlu_cloze:high_school_computer_science|0|1,custom|mmlu_cloze:high_school_european_history|0|1,custom|mmlu_cloze:high_school_geography|0|1,custom|mmlu_cloze:high_school_government_and_politics|0|1,custom|mmlu_cloze:high_school_macroeconomics|0|1,custom|mmlu_cloze:high_school_mathematics|0|1,custom|mmlu_cloze:high_school_microeconomics|0|1,custom|mmlu_cloze:high_school_physics|0|1,custom|mmlu_cloze:high_school_psychology|0|1,custom|mmlu_cloze:high_school_statistics|0|1,custom|mmlu_cloze:high_school_us_history|0|1,custom|mmlu_cloze:high_school_world_history|0|1,custom|mmlu_cloze:human_aging|0|1,custom|mmlu_cloze:human_sexuality|0|1,custom|mmlu_cloze:international_law|0|1,custom|mmlu_cloze:jurisprudence|0|1,custom|mmlu_cloze:logical_fallacies|0|1,custom|mmlu_cloze:machine_learning|0|1,custom|mmlu_cloze:management|0|1,custom|mmlu_cloze:marketing|0|1,custom|mmlu_cloze:medical_genetics|0|1,custom|mmlu_cloze:miscellaneous|0|1,custom|mmlu_cloze:moral_disputes|0|1,custom|mmlu_cloze:moral_scenarios|0|1,custom|mmlu_cloze:nutrition|0|1,custom|mmlu_cloze:philosophy|0|1,custom|mmlu_cloze:prehistory|0|1,custom|mmlu_cloze:professional_accounting|0|1,custom|mmlu_cloze:professional_law|0|1,custom|mmlu_cloze:professional_medicine|0|1,custom|mmlu_cloze:professional_psychology|0|1,custom|mmlu_cloze:public_relations|0|1,custom|mmlu_cloze:security_studies|0|1,custom|mmlu_cloze:sociology|0|1,custom|mmlu_cloze:us_foreign_policy|0|1,custom|mmlu_cloze:virology|0|1,custom|mmlu_cloze:world_religions|0|1"

In [ ]:
!lighteval accelerate --model_args "pretrained=LoserCheems/Doge-25M" --tasks "leaderboard|truthfulqa:mc|0|0" --override_batch_size 1 --output_dir="./evals/"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("LoserCheems/Doge-25M")
model = AutoModelForCausalLM.from_pretrained("LoserCheems/Doge-25M", trust_remote_code=True)
inputs = tokenizer("Hello", return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=2)
print(tokenizer.batch_decode(out))

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("S:\datasets\cosmopedia-v2_tokenized")
print(dataset)